In [1]:

import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

import numpy as np
from tqdm import tqdm
import torch

from datasets import load_dataset
from src.dataset import Dataset
from src.model_dump import Dump
from src.model_phase import PhraseModel
from src.model_query import QueryModel

raw_datasets = load_dataset("squad")

/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = Dataset(model_checkpoint='SpanBERT/spanbert-base-cased')
dataset.load_ds(raw_datasets, size=100)

100%|██████████| 100/100 [00:00<00:00, 1261.74it/s]


In [3]:
# lens = []
# for i in range(len(dataset.contexts)):
#     context_ids = dataset.tokenizer(dataset.contexts[i])['input_ids'][1:-1]
#     answer_ids = dataset.tokenizer(dataset.answers[i][0])['input_ids'][1:-1]
#     # print(dataset.contexts[i])
#     # print(dataset.answers[i][0])
#     # print(context_ids)
#     # print(answer_ids)
#     # start = context_ids.index(answer_ids[0]) 
#     # end = context_ids.index(answer_ids[-1])
    
#     start = dataset.spans_input_ids[i]['start']
#     end = dataset.spans_input_ids[i]['end']
#     print(len(context_ids), start, end)
#     print(dataset.tokenizer.decode(context_ids[start:end]), end='\n\n')

In [4]:
   
phrasemodel = PhraseModel(hidden_dim=768, ds=dataset, model_checkpoint='SpanBERT/spanbert-base-cased', device='cpu')
optimizer = torch.optim.Adam(phrasemodel.parameters(), lr=0.01)
L = len(dataset.contexts)
indices = list(range(L))
np.random.shuffle(indices)
batch_size = 5
inputs_batched = [indices[i*batch_size:(i+1)*batch_size]
                          for i in range(L //batch_size)]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
num_epochs = 5
for epoch in range(num_epochs):
    # Forward pass
    losses = []
    for inputs in tqdm(inputs_batched):
        optimizer.zero_grad()
        loss = phrasemodel(inputs)

        # # Backward and optimize
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    print(np.mean(losses))

  0%|          | 0/20 [00:00<?, ?it/s]/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/src/model_phase.py:50: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  z_start = torch.matmul(last_hidden_state[n], q_start[n].T).reshape(context_num_tokens)
/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


17.23252182006836


100%|██████████| 20/20 [00:39<00:00,  1.98s/it]


16.988368654251097


100%|██████████| 20/20 [00:37<00:00,  1.89s/it]


16.988368797302247


100%|██████████| 20/20 [00:39<00:00,  1.98s/it]


16.988368797302247


100%|██████████| 20/20 [00:40<00:00,  2.05s/it]

16.988368701934814


In [6]:
dump = Dump(dataset, phrasemodel.model, 
              phrasemodel.model_1, phrasemodel.model_2,
              phrasemodel.tokenizer, hidden_dim=768, 
              device='cpu')

dump.create_dump()

Creating Phrase dump: 100%|██████████| 21/21 [00:05<00:00,  4.20it/s]


In [5]:
dump.evaluate()

100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


{'em_string': 0.0,
 'em_start': 0.01,
 'em_end': 0.0,
 'delta_start_normalized': 0.059660774458420995,
 'delta_end_normalized': 0.061106907517283086}

In [7]:
dump.evaluate()

 74%|███████▍  | 74/100 [04:58<01:42,  3.94s/it]

In [ ]:
query_side_model =  QueryModel(phrasemodel.model, 
              phrasemodel.model_1, phrasemodel.model_2,
              hidden_dim=768, H=dump.H, ds=dataset,
              device='cpu')

In [ ]:
query_side_optimizer = torch.optim.Adam(query_side_model.parameters(), lr=0.01)

num_epochs = 5
for epoch in range(num_epochs):
    # Forward pass
    losses = []
    for inputs in tqdm(inputs_batched):
        query_side_optimizer.zero_grad()
        loss = query_side_model(inputs)

        # # Backward and optimize
        loss.backward()
        query_side_optimizer.step()
        losses.append(loss.item())
        
    print(np.mean(losses))

In [6]:
for i in range(100):
    answer, start_index, end_index, score = dump.predict(i, k=100, verbose=True)
    print(dataset.spans_input_ids[i]['start'], start_index)
    print(dataset.spans_input_ids[i]['end'], end_index)
    print(answer)
    print()

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
128 40
136 40


Q: What is in front of the Notre Dame Main Building?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of th

KeyboardInterrupt: 